In [1]:
import numpy as np
import os
import psycopg2
import pandas as pd
from datetime import datetime
import json

In [2]:
conncommend = "host='localhost' port=2345 dbname='expdata' user='postgres' password='psql123'"
conn = psycopg2.connect(conncommend)

In [21]:
dfile = 'D:\\Jun\\lab_data\\data\\EP\\single_TG_neuron_ep\\CSD_neuron_response.csv'
cfile = 'D:\\Jun\\lab_data\\project\\CSD induced electrophysiology responses\\CSD_and_treatment_log.xlsx'
rtfile = 'D:\\Jun\\lab_data\\data\\EP\\single_TG_neuron_ep\\raw\\CSD_neuron_response_trend.csv'
mtfile = 'D:\\Jun\\lab_data\\data\\EP\\single_TG_neuron_ep\\raw\\CSD_neuron_mech_trendline.csv'
tfile = 'C:\\Users\\jzhao1\\Downloads\\operation_method.xlsx'
root = 'D:\expdata\EP'

In [35]:
ddf = pd.read_csv(dfile)
cdf = pd.read_excel(cfile)
rdf = pd.read_csv(rtfile)
mdf = pd.read_csv(mtfile)
tdf = pd.read_excel(tfile)

# As the data structure is so complicated, I will import raw data first. I will set fileid as neuronid + last 2 of CSDID.

In [59]:
for i in range(len(mdf)):
    mdf.loc[i, 'fileid'] = mdf.loc[i, 'neuronID'] + mdf.loc[i, 'CSDID'][-2:]

In [60]:
tmplist = []
for i in range(len(mdf)):
    if mdf.loc[i, 'fileid'] not in tmplist:
        tmplist = np.append(tmplist, mdf.loc[i, 'fileid'])

In [61]:
tmplist[1][0:11]

'N2014032801'

In [63]:
for i in range(len(tmplist)):
    mechdata = mdf.loc[mdf.loc[:, 'fileid'] == tmplist[i], :].sort_values(by = 'time_point').reset_index(drop = True)
    mech_dataid = 'mech'+tmplist[i]
    treat_pt_mech = sum([('B' in x) for x in mechdata.loc[:, 'time_point'].values])
    tmp = np.concatenate((np.reshape(mechdata.loc[:, 'TH_value'].values, [1, -1]), 
                          np.reshape(mechdata.loc[:, 'STH_value'].values, [1, -1])), axis = 0)
    shape = list2array(np.shape(tmp))
    print(shape)
    print('==============================')
            
    
    np.save(os.path.join(root, 'SINGLEMECH', mech_dataid+'.npy'), tmp)

    cur = conn.cursor()
    cur.execute(
        """
        INSERT INTO data_ep (id, rate, shape, ref_path, type)
        values ('{}', 0.001111, '{}', '{}', 'SINGLEMECH')
        """.format(mech_dataid, shape, 'SINGLEMECH\\'+mech_dataid+'.npy')
    )
    conn.commit()
    
    

{2, 12}
{2, 9}
{2, 19}
{2, 14}
{2, 14}
{2, 12}
{2, 12}
{2, 13}
{2, 11}
{2, 14}
{2, 10}
{2, 10}
{2, 11}
{2, 10}
{2, 14}
{2, 10}
{2, 11}
{2, 10}
{2, 11}
{2, 12}
{2, 12}
{2, 15}
{2, 15}
{2, 10}
{2, 12}
{2, 13}
{2, 19}
{2, 10}
{2, 10}
{2, 10}
{2, 10}
{2, 13}
{2, 9}
{2, 11}
{2, 14}
{2, 14}
{2, 13}
{2, 18}
{2, 15}
{2, 10}
{2, 10}
{2, 10}
{2, 15}
{2, 12}
{2, 12}
{2, 9}
{2, 13}
{2, 11}
{2, 10}
{2, 18}
{2, 10}
{2, 17}
{2, 9}
{2, 16}
{2, 12}
{2, 12}
{2, 9}
{2, 13}
{2, 12}
{2, 13}
{2, 11}
{2, 11}
{2, 14}
{2, 10}
{2, 13}
{2, 9}
{2, 13}
{2, 10}
{2, 10}
{2, 14}
{2, 15}
{2, 13}
{2, 12}
{2, 12}
{2, 14}
{2, 13}
{2, 18}
{2, 14}
{2, 14}
{2, 12}
{2, 13}
{2, 14}
{2, 14}
{2, 9}
{2, 9}
{2, 9}
{2, 12}
{2, 10}
{2, 12}
{2, 7}
{2, 9}
{2, 19}
{2, 16}
{2, 13}
{2, 13}
{2, 15}
{2, 16}
{2, 14}
{2, 13}
{2, 13}
{2, 8}
{2, 13}
{2, 15}
{2, 14}
{2, 13}
{2, 15}
{2, 13}
{2, 13}
{2, 16}
{2, 13}
{2, 14}
{2, 11}
{2, 13}
{2, 14}
{2, 13}
{2, 16}
{2, 14}
{2, 15}
{2, 16}
{2, 15}
{2, 16}
{2, 14}
{2, 17}
{2, 13}
{2, 16}
{2, 9}
{2, 1

<h1> produce treatment json file

In [15]:
tdic = {}
for i in range(len(tdf)):
    tdic[tdf.loc[i, 'treatID']] = {'method': tdf.loc[i, 'drug_name'], 
                                   'concentration': tdf.loc[i, 'concentration'],
                                   'apply_method': tdf.loc[i, 'administration_method'],
                                   'dose': tdf.loc[i, 'volumn']
                                  }

In [18]:
with open('C:\\Users\\jzhao1\\Downloads\\t.json', 'w') as outfile:  
    json.dump(tdic, outfile)


In [64]:
tdic

{'TID00001': {'method': 'A10606120-expire-1',
  'concentration': '0.1mM',
  'apply_method': 'topic',
  'dose': nan},
 'TID00002': {'method': 'A10606120-expire',
  'concentration': '0.1mM',
  'apply_method': 'topic',
  'dose': nan},
 'TID00003': {'method': 'A10606120',
  'concentration': '0.1mM',
  'apply_method': 'topic',
  'dose': nan},
 'TID00004': {'method': 'A10606120',
  'concentration': '10um',
  'apply_method': 'topic',
  'dose': nan},
 'TID00005': {'method': 'ACSF',
  'concentration': nan,
  'apply_method': 'icv',
  'dose': '10ul'},
 'TID00006': {'method': 'ACSF',
  'concentration': nan,
  'apply_method': 'icv',
  'dose': '2ul'},
 'TID00007': {'method': 'ACSF',
  'concentration': nan,
  'apply_method': 'icv',
  'dose': '5ul'},
 'TID00008': {'method': 'AL8810-',
  'concentration': '1mM',
  'apply_method': 'topic',
  'dose': nan},
 'TID00009': {'method': 'AL8810',
  'concentration': '1mM',
  'apply_method': 'topic',
  'dose': nan},
 'TID00010': {'method': 'BIBN',
  'concentration

# functions

In [28]:
def list2array(list):
    tmp = str(list).replace('(', '{').replace(')', '}').replace('[', '{').replace(']', '}').replace('\'', '\"')
    return(tmp)

def cnoprotocol(idx):
    if idx in [1,3]:
        method = {'method': 'CNO', 'apply_method': 'ip', 'concentration': '1mg/cc', 'dose': '1mg/cc'}
    elif idx == 2:
        method = {'method': 'CNO', 'apply_method': 'topic', 'concentration': '0.5mM'}
    elif idx == 4:
        method = {'method': 'compound21', 'apply_method': 'ip', 'concentration': '1mg/cc', 'dose': '1mg/cc'}
    return(method)
def cellfillnan(value):
    if pd.isna(value):
        return('')
    else:
        return(value)

In [8]:
# researcher data note date projectid animalid id results

In [29]:
finished_exp = []

In [65]:
finished_exp

[]

In [26]:
ddf

,neuronID,CSDID,pinprick_activation,neuron_immediate_activation,neuron_delay_activation,neuron_delay_activation_lasting,neuron_sensitization_force1_whe,neuron_sensitization_force1_lasting,neuron_sensitization_force2_whe,neuron_sensitization_force2_lasting,neuron_activation_baseline_average,neuron_activation_baseline_CI,force1_baseline,force1_ci,force2_baseline,force2_ci,note
0,N2013040101,C130401011,N,Y,Y,45.0,NaN,NaN,NaN,NaN,0.010000,0.007938,NaN,NaN,NaN,NaN,NaN
1,N2013040201,C130402011,N,N,Y,35.0,NaN,NaN,NaN,NaN,1.610000,0.029391,NaN,NaN,NaN,NaN,NaN
2,N2013040301,C130403011,N,N,Y,10.0,NaN,NaN,NaN,NaN,0.010000,0.007269,NaN,NaN,NaN,NaN,NaN
3,N2013040501,C130405011,N,Y,Y,40.0,NaN,NaN,NaN,NaN,0.729394,0.207294,NaN,NaN,NaN,NaN,NaN
4,N2013041501,C130415011,N,Y,Y,55.0,NaN,NaN,NaN,NaN,0.452576,0.026093,NaN,NaN,NaN,NaN,NaN
5,N2013042201,C130422011,Y,Y,N,NaN,NaN,NaN,NaN,NaN,1.206364,0.037964,NaN,NaN,NaN,NaN,NaN
6,N2013061701,C130617011,N,N,N,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN
7,N2013061801,C130618011,N,N,N,NaN,NaN,NaN,NaN,NaN,5.073333,0.404084,NaN,NaN,NaN,NaN,NaN
8,N2013061901,C130619011,N,Y,Y,40.0,NaN,NaN,NaN,NaN,0.054091,0.020918,NaN,NaN,NaN,NaN,NaN
9,N2013062001,C130620011,N,Y,N,NaN,NaN,NaN,NaN,NaN,0.382222,0.118941,NaN,NaN,NaN,NaN,"the baseline of this neuron is pretty high, bu..."


In [107]:
for i in range(10):#1, len(ddf)):
    
    expid = ddf.loc[i, 'neuronID']+'_'+ddf.loc[i, 'CSDID'][-2:]
    
    if expid not in finished_exp:
        note = ddf.loc[i, 'note']
        neuronid = ddf.loc[i, 'neuronID']
        date = datetime.strptime(neuronid[1:-2], '%Y%m%d')
        animalid = 'A'+neuronid[1:-1]+ddf.loc[i, 'CSDID'][-2]

        ongoingdata = rdf.loc[rdf.loc[:, 'fileid'] == expid, :].sort_values(by = 'time_point').reset_index(drop = True)
        ongoing_dataid = 'ongoing'+neuronid
        treat_pt_ongo = sum([('B' in x) for x in ongoingdata.loc[:, 'time_point'].values])
        tmp = ongoingdata.loc[:, 'record_value'].values
        shape = list2array(np.shape(np.reshape(tmp, [1, -1])))
        """
        np.save(os.path.join(root, 'SINGLESPON', ongoing_dataid+'.npy'), tmp)


        cur = conn.cursor()
        cur.execute(

            INSERT INTO data_ep (id, rate, shape, ref_path, type)
            values ('{}', 0.003333, '{}', '{}', 'SINGLESPON')
            .format(ongoing_dataid, shape, 'SINGLESPON\\'+ongoing_dataid+'.npy')
        )
        conn.commit()
        """


        mechdata = mdf.loc[mdf.loc[:, 'neuronID'] == neuronid, :].sort_values(by = 'time_point').reset_index(drop = True)
        mech_dataid = 'mech'+neuronid
        treat_pt_mech = sum([('B' in x) for x in mechdata.loc[:, 'time_point'].values])
        tmp = np.concatenate((np.reshape(mechdata.loc[:, 'TH_value'].values, [1, -1]), 
                              np.reshape(mechdata.loc[:, 'STH_value'].values, [1, -1])), axis = 0)
        shape = list2array(np.shape(tmp))
        """
        np.save(os.path.join(root, 'SINGLEMECH', mech_dataid+'.npy'), tmp)

        cur = conn.cursor()
        cur.execute(

            INSERT INTO data_ep (id, rate, shape, ref_path, type)
            values ('{}', 0.003333, '{}', '{}', 'SINGLEMECH')
            .format(mech_dataid, shape, 'SINGLEMECH\\'+mech_dataid+'.npy')
        )
        conn.commit()
        """


        data = {}
        data['TGneuron_ongoing'] = {'dataid': ongoing_dataid, 'treatment':{'0':0, '1':treat_pt_ongo}}
        data['TGneuron_mech'] = {'dataid': mech_dataid, 'treatment': {'0':0, '1': treat_pt_mech}}
        treatmethod = cnoprotocol(ddf.loc[i, 'CNO_protocol'])
        data['treatment'] = {'0': {'method': 'baseline'}, '1': treatmethod}
        print(data)

        print('===========================')

        result = {}
        result['TGneuron_ongoing'] = {'delay_activation': cellfillnan(ddf.loc[i, 'neuron_delay_activation']), 
                                      'delay_activation_lasting': cellfillnan(ddf.loc[i, 'neuron_delay_activation_lasting']),
                                      'activation_baseline_ave': cellfillnan(ddf.loc[i, 'neuron_activation_baseline_average']),
                                      'activation_baseline_ci': cellfillnan(ddf.loc[i, 'neuron_activation_baseline_CI'])
                                     }
        result['TGneuron_mech'] = {'th_sensitization': cellfillnan(ddf.loc[i, 'neuron_sensitization_force1_whe']),
                                   'th_sensitization_lasting': cellfillnan(ddf.loc[i, 'neuron_sensitization_force1_lasting']),
                                   'th_baseline': cellfillnan(ddf.loc[i, 'force1_baseline']),
                                   'th_ci': cellfillnan(ddf.loc[i, 'force1_ci']),
                                   'sth_sensitization': cellfillnan(ddf.loc[i, 'neuron_sensitization_force2_whe']),
                                   'sth_sensitization_lasting': cellfillnan(ddf.loc[i, 'neuron_sensitization_force2_lasting']),
                                   'sth_baseline': cellfillnan(ddf.loc[i, 'force2_baseline']),
                                   'sth_ci': cellfillnan(ddf.loc[i, 'force2_ci'])
                                  }

        cur = conn.cursor()
        cur.execute(
            """
            INSERT INTO exp_ep_invivo_anes (researcher, data, note, date, projectid, animalid, id, results)
            values ('{}', '{}', '{}', '{}', 3, '{}',{}, '{}')
            """.format(list2array(['Jun Zhao']), json.dumps(data), note, date, animalid, int(i+5), json.dumps(result))
        )
        conn.commit()
        
        finished_exp = np.append(finished_exp, expid)
        print('%s done' % expid)
    
    else:
        print('%s is already finished' % expid)
    
    
    
    

{'TGneuron_ongoing': {'dataid': 'ongoingN2015121601', 'treatment': {'0': 0, '1': 12}}, 'TGneuron_mech': {'dataid': 'mechN2015121601', 'treatment': {'0': 0, '1': 4}}, 'treatment': {'0': {'method': 'baseline'}, '1': {'method': 'CNO', 'apply_method': 'ip', 'concentration': '1mg/cc', 'dose': '1mg/cc'}}}
{'TGneuron_ongoing': {'dataid': 'ongoingN2015121701', 'treatment': {'0': 0, '1': 12}}, 'TGneuron_mech': {'dataid': 'mechN2015121701', 'treatment': {'0': 0, '1': 3}}, 'treatment': {'0': {'method': 'baseline'}, '1': {'method': 'CNO', 'apply_method': 'ip', 'concentration': '1mg/cc', 'dose': '1mg/cc'}}}
{'TGneuron_ongoing': {'dataid': 'ongoingN2016040601', 'treatment': {'0': 0, '1': 12}}, 'TGneuron_mech': {'dataid': 'mechN2016040601', 'treatment': {'0': 0, '1': 4}}, 'treatment': {'0': {'method': 'baseline'}, '1': {'method': 'CNO', 'apply_method': 'ip', 'concentration': '1mg/cc', 'dose': '1mg/cc'}}}
{'TGneuron_ongoing': {'dataid': 'ongoingN2016040701', 'treatment': {'0': 0, '1': 12}}, 'TGneuron_

In [71]:
ddf.loc[0, :]

neuronID                                                                     N2015121501
CNO_protocol                                                                           1
neuron_delay_activation                                                                Y
neuron_delay_activation_lasting                                                       10
neuron_sensitization_force1_whe                                                        Y
neuron_sensitization_force1_lasting                                                  120
neuron_sensitization_force2_whe                                                        Y
neuron_sensitization_force2_lasting                                                  105
neuron_activation_baseline_average                                              0.900833
neuron_activation_baseline_CI                                                   0.116073
force1_baseline                                                                  6.16667
force1_ci            

In [98]:
cur = conn.cursor()
cur.execute('rollback;')
conn.commit()

In [83]:
json.dumps(data)

TypeError: Object of type 'int32' is not JSON serializable

In [84]:
data

{'TGneuron_ongoing': {'dataid': 'ongoingN2015121501',
  'treatment': {'0': 0, '1': 12}},
 'TGneuron_mech': {'dataid': 'mechN2015121501', 'treatment': {'0': 0, '1': 4}},
 'treatment': {'0': {'method': 'baseline'},
  '1': {'method': 'CNO',
   'apply_method': 'ip',
   'concentration': '1mg/cc',
   'dose': '1mg/cc'}}}

In [85]:
json.dumps({'0':100})

'{"0": 100}'